<a href="https://colab.research.google.com/github/parvathysarat/gpt2-text-generation/blob/master/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install transformers
!git clone https://github.com/huggingface/transformers
os.chdir('transformers')
print(os.getcwd())
!pip install .

     |████████████████████████████████| 778kB 4.9MB/s 
     |████████████████████████████████| 1.1MB 28.3MB/s 
     |████████████████████████████████| 3.0MB 40.0MB/s 
     |████████████████████████████████| 890kB 54.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=c97418360ffe92bf911645ceec1bd812d26b7d58221b42de3e7e9195d35edd1a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Cloning into 'transformers'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 38349 (delta 9), reused 9 (delta 2), pack-reused 38327
Receiving objects: 100% (38349/38349), 27.74 MiB | 26.13 MiB/s, done.
Resolving deltas: 100% (26569/26569), done.


In [13]:
import os
os.chdir('examples/language-modeling/')

## Fine tuning GPT-2 language model on our dataset

Add to transformers/example/language-modeling/run-language-modelling.py file the following snippet : <br>

```
special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
```


In [37]:
N=1
try:os.mkdir('../../../model')
except:pass
OUTPUT_DIR='../../../model/'
print(os.getcwd())
TRAIN_FILE='../../../dataset/train.txt'
VALID_FILE='../../../dataset/valid.txt'


/content/transformers/examples/language-modeling


In [39]:
os.environ["CUDA_VISIBLE_DEVICES"]= "1"
!python run_language_modeling.py \
--output_dir=$OUTPUT_DIR \
--model_type=gpt2 \
--model_name_or_path=gpt2 \
--do_train \
--train_data_file=$TRAIN_FILE \
--do_eval \
--eval_data_file=$VALID_FILE \
--per_device_train_batch_size=2 \
--per_device_eval_batch_size=2 \
--line_by_line \
--evaluate_during_training \
--learning_rate=5e-5 \
--num_train_epochs=5 \
--overwrite_output_dir

Streaming output truncated to the last 5000 lines.

Evaluation:  59% 812/1376 [03:42<03:28,  2.71it/s]

Evaluation:  59% 813/1376 [03:42<03:09,  2.96it/s]

Evaluation:  59% 814/1376 [03:43<03:01,  3.09it/s]

Evaluation:  59% 815/1376 [03:43<03:08,  2.98it/s]

Evaluation:  59% 816/1376 [03:43<02:38,  3.53it/s]

Evaluation:  59% 817/1376 [03:43<02:22,  3.93it/s]

Evaluation:  59% 818/1376 [03:44<02:15,  4.11it/s]

Evaluation:  60% 819/1376 [03:44<02:01,  4.57it/s]

Evaluation:  60% 820/1376 [03:44<01:51,  4.99it/s]

Evaluation:  60% 821/1376 [03:44<01:50,  5.01it/s]

Evaluation:  60% 822/1376 [03:44<01:43,  5.37it/s]

Evaluation:  60% 823/1376 [03:45<02:02,  4.50it/s]

Evaluation:  60% 824/1376 [03:45<01:43,  5.31it/s]

Evaluation:  60% 825/1376 [03:45<02:24,  3.82it/s]

Evaluation:  60% 826/1376 [03:45<02:02,  4.49it/s]

Evaluation:  60% 827/1376 [03:45<01:51,  4.94it/s]

Evaluation:  60% 828/1376 [03:46<01:46,  5.16it/s]

Evaluation:  60% 829/1376 [03:46<02:25,  3.76it/s]

Evaluation: 

## Evaluate perplexity on test set

In [42]:
TEST_FILE='../../../dataset/valid.txt'
CUDA_VISIBLE_DEVICES="1" 
!python run_language_modeling.py \
--output_dir=$OUTPUT_DIR \
--model_type=gpt2 \
--model_name_or_path=$OUTPUT_DIR \
--do_eval \
--eval_data_file=$TEST_FILE \
--per_device_eval_batch_size=2 \
--line_by_line

2020-08-16 07:38:24.140171: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
08/16/2020 07:38:25 - INFO - transformers.training_args -   PyTorch: setting up devices
08/16/2020 07:38:25 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
08/16/2020 07:38:25 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../model/', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluate_during_training=False, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Aug16_07-38-25_39510ae949d6', loggi

## Text generation

Setting prompt text: "I always wonder"

Using top-k sampling decoder from GPT-2 (k=50 has been proven to be more effective at generating non-repetitive texts.) Other decoders are also available.

In [ ]:
prompt_text = "I always wonder"
encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=True, return_tensors=”pt”)

In [ ]:
os.chdir('../text-generation')
K=50
CUDA_VISIBLE_DEVICES="1" python run_generation.py \
--model_type gpt2 \
--model_name_or_path $OUTPUT_DIR \
--length 300 \
--prompt "<BOS>" \
--stop_token "<EOS>" \
--k $K \
--num_return_sequences 5